Iniciando a uniação e validação das bases de dados extraidas

In [35]:
import pandas as pd

pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

In [ ]:
#Games 
df_list = []
for i in range(1, 8):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

# df_1 = pd.read_excel(path + r'\01_games.xlsx')
# df_2 = pd.read_excel(path + r'\02_games.xlsx')
# df_3 = pd.read_excel(path + r'\03_games.xlsx')
# df_4 = pd.read_excel(path + r'\04_games.xlsx')
# df_5 = pd.read_excel(path + r'\05_games.xlsx')
# df_6 = pd.read_excel(path + r'\06_games.xlsx')
# df_7 = pd.read_excel(path + r'\07_games.xlsx')

Visão geral de cada um das bases de dados, escolha dos principais atributos para avaliar público.

DF2 removido, não tem identificação explicita de partida, ano! da pra fazer mas as outras bases de dados já são suficientes.

In [45]:
db_id = 1  # 0 - 6
display(df_list[db_id].describe())
display(df_list[db_id].head(-3))
display(df_list[db_id].info())

,ID,rodata,mandante_Placar,visitante_Placar
count,8405.00000,8405.000000,8405.000000,8405.000000
mean,4203.00000,20.133254,1.541225,1.027008
std,2426.45884,11.444643,1.230607,1.027170
min,1.00000,1.000000,0.000000,0.000000
25%,2102.00000,10.000000,1.000000,0.000000
50%,4203.00000,20.000000,1.000000,1.000000
75%,6304.00000,30.000000,2.000000,2.000000
max,8405.00000,46.000000,7.000000,7.000000


,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,1,29/03/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
1,2,1,29/03/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
2,3,1,30/03/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
3,4,1,30/03/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
4,5,1,30/03/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8397,8398,38,06/12/2023,21:32,Coritiba,Corinthians,4-2-3-1,4-3-3,A. Ferreira,L. Venker de Menezes,Corinthians,Couto Pereira,0,2,PR,SP
8398,8399,38,06/12/2023,21:32,Fluminense,Gremio,4-2-3-1,4-3-3,F. Diniz Silva,Renato Gaúcho,Gremio,Maracanã,2,3,RJ,RS
8399,8400,38,06/12/2023,21:32,Cruzeiro,Palmeiras,4-3-3,3-4-1-2,P. Autuori de Mello,A. Moreira Ferreira,-,Mineirão,1,1,MG,SP
8400,8401,38,06/12/2023,21:32,Bahia,Atletico-MG,3-4-2-1,4-4-2,R. Ceni,L. Scolari,Bahia,Itaipava Arena Fonte Nova,4,1,BA,MG


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405 entries, 0 to 8404
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  8405 non-null   int64 
 1   rodata              8405 non-null   int64 
 2   data                8405 non-null   object
 3   hora                8405 non-null   object
 4   mandante            8405 non-null   object
 5   visitante           8405 non-null   object
 6   formacao_mandante   3431 non-null   object
 7   formacao_visitante  3431 non-null   object
 8   tecnico_mandante    3795 non-null   object
 9   tecnico_visitante   3795 non-null   object
 10  vencedor            8405 non-null   object
 11  arena               8405 non-null   object
 12  mandante_Placar     8405 non-null   int64 
 13  visitante_Placar    8405 non-null   int64 
 14  mandante_Estado     8405 non-null   object
 15  visitante_Estado    8405 non-null   object
dtypes: int64(4), object(12)


None

Padronizar data, mandante e visitante para indexar arquivos e agrupalos

In [47]:
mapeamento_colunas = {
    'data': 'Data',
    'Date': 'Data',
    'data_partida': 'Data',
    'time_mandante': 'Mandante',
    'madante': 'Mandante',
    'Home': 'Mandante',
    'time_visitante': 'Visitante',
    'visitante': 'Visitante',
    'Away': 'Visitante',
}

df_list_map = [df.rename(columns=mapeamento_colunas) for df in df_list]

for i in range(len(df_list)):
    df_list_map[i].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8453 entries, 0 to 8452
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ano_campeonato                  8453 non-null   int64  
 1   Data                            8453 non-null   object 
 2   rodada                          8453 non-null   int64  
 3   estadio                         8442 non-null   object 
 4   arbitro                         6743 non-null   object 
 5   publico                         6742 non-null   float64
 6   publico_max                     4181 non-null   float64
 7   Mandante                        8453 non-null   object 
 8   Visitante                       8453 non-null   object 
 9   tecnico_mandante                6299 non-null   object 
 10  tecnico_visitante               6299 non-null   object 
 11  colocacao_mandante              6743 non-null   float64
 12  colocacao_visitante             67